In [1]:
import requests as r
import os
from bs4 import BeautifulSoup as bs
import lxml
from urllib.parse import urlparse,parse_qs,urlencode,urlunparse
import sqlite3
import re

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    # 如果目标站点需要，可以加上 Cookie 或 Referer
    # "Referer": "https://example.com/",
    # "Cookie": "name=value; name2=value2"
}

def fetch_url(url,file=None):
    respond=r.get(url,headers=headers)
    if respond.status_code==200:
        if file:
            with open(file, "w") as f:
                f.write(respond.text)
            print(f"File saved in {file}.")
        return respond
    else:
        raise Exception(f"Failed to download file. Status code: {respond.status_code}")


if __name__=="__main__":
    
    url="https://wiki.biligame.com/blhx/首页"
    index_file="html/index.html"
    bg_index_file="html/bg.html"   

    if not os.path.exists(index_file):
        fetch_url(url,index_file)
    index_soup=bs(open("html/index.html"),"lxml")
    a_list=index_soup.find_all("a",class_="menu-title-1")
    bg=[_ for _ in a_list if "壁纸和插画" in _.text][0]
    parsed=urlparse(url)
    bg_url=urlunparse(parsed._replace(path=bg["href"]))

    if not os.path.exists(bg_index_file):
        fetch_url(bg_url,bg_index_file)
    bg_soup=bs(open(bg_index_file),"lxml")

    conn = sqlite3.connect('images.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS images
                (Id INTEGER PRIMARY KEY AUTOINCREMENT,
                URL TEXT,
                title TEXT,
                artist TEXT,
                original_url TEXT)''')

    current_shared_title = None

    for li in bg_soup.select('li.gallerybox'):
        thumb_div = li.select_one('div.thumb')
        if thumb_div:
            a_tag = thumb_div.find('a', class_='image')
            if a_tag and 'href' in a_tag.attrs:
                href = a_tag['href']
            else:
                href = None
        else:
            href = None
        
        # 处理gallerytext部分
        gallerytext = li.select_one('div.gallerytext')
        title = None
        artist = None
        
        if gallerytext:
            p_tag = gallerytext.find('p')
            if p_tag:
                if not p_tag.find('a'):
                    title = p_tag.get_text(strip=True)
                    current_shared_title = title  # 设置共享标题
                else:
                    artist = p_tag.get_text(strip=True)
                    current_shared_title = None  # 重置共享标题
            elif current_shared_title:
                title = current_shared_title
        
        parsed=urlparse(url)
        href=urlunparse(parsed._replace(path=href))
        c.execute('''INSERT INTO images (URL, title, artist, original_url)
                    VALUES (?, ?, ?, NULL)''', 
                    (href, title, artist))

    conn.commit()
    conn.close()

In [ ]:
import os
import sqlite3
import requests
from bs4 import BeautifulSoup
import re
import time
import hashlib

# 配置信息
DATABASE_PATH = "images.db"  # SQLite数据库路径
DOWNLOAD_DIR = "downloads"  # 下载根目录
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    # 如果目标站点需要，可以加上 Cookie 或 Referer
    # "Referer": "https://example.com/",
    # "Cookie": "name=value; name2=value2"
}
REQUEST_DELAY = 1  # 请求延迟（秒），避免请求过快

def ensure_directory_exists(path):
    """确保目录存在，不存在则创建"""
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"创建目录: {path}")

def download_file(url, save_path):
    """下载文件并保存到指定路径"""
    try:
        response = requests.get(url, headers=HEADERS, stream=True, timeout=30)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"下载失败: {url} - {str(e)}")
        return False

def sanitize_filename(name):
    """清理文件名中的非法字符"""
    return re.sub(r'[\\/*?:"<>|]', '', name)

def generate_unique_filename(artist, original_filename, existing_files):
    """生成唯一的文件名，避免重复"""
    base_name = f"{artist}_{original_filename}"
    if base_name not in existing_files:
        return base_name
    
    # 如果文件名已存在，添加哈希后缀
    name, ext = os.path.splitext(original_filename)
    hash_suffix = hashlib.md5(artist.encode()).hexdigest()[:6]
    return f"{artist}_{name}_{hash_suffix}{ext}"

def process_database():
    """处理数据库中的记录，下载原始文件并更新数据库"""
    # 连接数据库
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    
    # 添加original_url列（如果不存在）
    cursor.execute("PRAGMA table_info(images)")
    columns = [col[1] for col in cursor.fetchall()]
    if "original_url" not in columns:
        cursor.execute("ALTER TABLE images ADD COLUMN original_url TEXT")
        print("已添加original_url列")
    
    # 获取需要处理的记录（original_url为空的记录）
    cursor.execute("SELECT Id, URL, title, artist FROM images WHERE original_url IS NULL")
    records = cursor.fetchall()
    
    if not records:
        print("没有需要处理的记录")
        return
    
    total_records = len(records)
    print(f"找到 {total_records} 条需要处理的记录")
    
    # 创建下载根目录
    ensure_directory_exists(DOWNLOAD_DIR)
    
    # 创建文件夹"0"（用于存放没有标题但有艺术家的文件）
    folder_0 = os.path.join(DOWNLOAD_DIR, "0")
    ensure_directory_exists(folder_0)
    
    # 获取文件夹"0"中已有的文件列表（用于避免文件名冲突）
    existing_files = set()
    if os.path.exists(folder_0):
        existing_files = set(os.listdir(folder_0))
    
    # 处理每条记录
    for i, record in enumerate(records, 1):
        id_, url, title, artist = record
        
        print(f"\n处理记录 {i}/{total_records} (ID: {id_}): {url}")
        
        try:
            # 获取详情页内容
            response = requests.get(url, headers=HEADERS, timeout=15)
            response.raise_for_status()
            
            # 解析HTML
            soup = BeautifulSoup(response.text, 'lxml')
            
            # 查找原始文件链接
            full_media_div = soup.find('div', class_='fullMedia')
            if not full_media_div:
                print(f"未找到原始文件链接: {url}")
                continue
                
            original_link = full_media_div.find('a', class_='internal')
            if not original_link or 'href' not in original_link.attrs:
                print(f"未找到原始文件链接: {url}")
                continue
                
            original_url = original_link['href']
            print(f"找到原始文件URL: {original_url}")
            
            # 确定保存目录
            if title:
                folder_name = sanitize_filename(title.strip())
                save_dir = os.path.join(DOWNLOAD_DIR, folder_name)
                ensure_directory_exists(save_dir)
                
                # 从URL提取文件名
                file_name = os.path.basename(original_url)
                save_path = os.path.join(save_dir, file_name)
            else:
                # 没有标题的情况
                save_dir = folder_0
                
                # 从URL提取原始文件名
                original_filename = os.path.basename(original_url)
                
                # 如果有艺术家信息
                if artist:
                    artist_clean = sanitize_filename(artist.strip())
                    
                    # 生成唯一的文件名（避免重复）
                    file_name = generate_unique_filename(artist_clean, original_filename, existing_files)
                    
                    # 添加到现有文件列表（避免后续重复）
                    existing_files.add(file_name)
                else:
                    # 既没有标题也没有艺术家
                    file_name = original_filename
                
                save_path = os.path.join(save_dir, file_name)
            
            # 检查文件是否已存在
            if os.path.exists(save_path):
                print(f"文件已存在: {save_path}")
                # 更新数据库但不重新下载
                cursor.execute(
                    "UPDATE images SET original_url = ? WHERE Id = ?",
                    (original_url, id_)
                )
                conn.commit()
                print(f"记录 {id_} 更新成功")
                continue
            
            # 下载文件
            print(f"下载文件到: {save_path}")
            if download_file(original_url, save_path):
                # 更新数据库
                cursor.execute(
                    "UPDATE images SET original_url = ? WHERE Id = ?",
                    (original_url, id_)
                )
                conn.commit()
                print(f"记录 {id_} 更新成功")
            else:
                print(f"记录 {id_} 下载失败")
                
            # 请求延迟，避免过快请求
            time.sleep(REQUEST_DELAY)
                
        except Exception as e:
            print(f"处理记录 {id_} 时出错: {str(e)}")
    
    # 关闭数据库连接
    conn.close()
    print("\n处理完成")

if __name__ == "__main__":
    process_database()